# 좌표값으로 MET 값 예측
14개의 좌표(x,y)값을 활용하여 MET 값을 예측해보기 

Data : 100개 <br>
좌표 개수 : 14개


## --------------------------------------------------------------------------------------
## Data 구조 처리
학습용으로 사용하기 위한 이미지 데이터와 좌푯값 전처리 코드.

CSV 구성 형식
- 이미지 경로 , 관절좌표1, 관절좌표2, ... , 관절좌표14, MET Score

처리 형식
- 전체 데이터세트 이미지 최대 크기와 평균 크기 측정 -> 이미지 크기 조정
  크기 조정 비율에 따른 좌표값 재처리


In [1]:
import numpy as np # Numpy 배열 사용용도
import matplotlib.pyplot as plt # 이미지출력 용도
import pandas as pd # csv 파일을 pandas 프레임으로 호출 용도
import skimage
from skimage import io, transform # 이미지 불러오기 및 저장용도

# 주피터 노트북 상에 이미지를 출력하도록 설정
%matplotlib inline

In [2]:
# dataset CSV 파일을 불러와 pandas 프레임으로 저장한 뒤에 Numpy형식의 행렬로 변환하여 testset 변수에 저장
testset=pd.read_csv("/home/ejjchl77/predictMET/dataset.csv",header=None).as_matrix()

testset

array([['/var/data/MET/images/walking/20.bmp', '[17, 146]', '[23, 115]',
        ..., '[40, 39]', '[35, 17]', 0.7],
       ['/var/data/MET/images/walking/19.bmp', '[22, 129]', '[12, 116]',
        ..., '[21, 38]', '[14, 16]', 0.7],
       ['/var/data/MET/images/walking/18.bmp', '[23, 153]', '[21, 129]',
        ..., '[23, 41]', '[20, 21]', 0.7],
       ..., 
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/18.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[175, 61]', '[192, 1]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/19.jpg',
        '[-1, -1]', '[98, 170]', ..., '[172, 59]', '[201, 8]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/20.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[194, 55]', '[185, 8]', 1.8]], dtype=object)

In [3]:
testset.shape

(100, 16)

In [4]:
# 변수 초기화.
# 모든 x축 크기 총합, 모든 y축 크기 총합, 최대 x축 크기, 최대 y축 크기
allsum_x=allsum_y=max_x=max_y=0

for i in range(len(testset)): # testset 길이만큼 반복 // len(testset) = 100개 데이터
    img=skimage.io.imread(testset[i][0]) # 이미지 불러오기 (이미지 저장위치 불러오기)
    # img.shape의 구조는 "행렬의 rows, columns, channels의 갯수"
    
    if max_x<img.shape[0]: max_x=img.shape[0] # 최대 x축 길이값 저장
    if max_y<img.shape[1]: max_y=img.shape[1] # 최대 y축 길이값 저장
    allsum_x+=img.shape[0] # 모든 이미지의 x축 크기 총합
    allsum_y+=img.shape[1] # 모든 이미지의 y축 크기 총합
avg_x=allsum_x/len(testset) # 전체 이미지의 x축 크기 평균
avg_y=allsum_y/len(testset) # 전체 이미지의 y축 크기 평균
print (avg_x,avg_y,max_x,max_y) # x축 크기 평균, y축 크기 평균, 최대 x축 크기, 최대 y축 크기 출력


221.51 243.43 769 834


평균 : (x,y) =(221.51, 243.43)

이미지 크기를 256 x 256 으로 조정

In [5]:
# 문자열의 좌표값을 정수형 좌표값으로 변환하는 함수
def convert_str_to_int(coors):
    coor=np.zeros([len(coors),14,2],dtype=int) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(coors)): # 주어진 갯수 만큼 반복
        for j in range(14): # 좌표 개수 만큼 반복
            a=0 # 컴마 위치 확인용
            # "," 위치 체크
            while True: # 무한 반복문
                if coors[i][j+1][a]=="," : break # 해당 문자가 컴마면 무한 반복문 탈출
                else : a+=1 # 컴마가 아니면 다음 문자로 넘어감
            # ,를 기준으로 각각의 좌표 x y를 나누어 int로 형변환 하여 저장
            coor[i][j]=int(coors[i][j+1][1:a]),int(coors[i][j+1][a+1:-1])
            
    # 이렇게 반복하여 추출한 좌표값을 반환
    return coor


In [7]:
def convert2_str_to_float(dataset):
    MET=np.zeros([len(dataset),1],dtype=float) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(dataset)): # 주어진 갯수 만큼 반복
        a=0 # ']' 위치 확인용
        # "]" 위치 체크
        #while True: # 무한 반복문
         #   j=15
         #   if dataset[i][j][a]=="]" : break # 해당 문자가 ']'면 무한 반복문 탈출
         #   else : a+=1 # ]가 아니면 다음 문자로 넘어감
         #   # ]를 기준으로 MET값을 int로 형변환 하여 저장
        MET[i]=float(dataset[i][15])
       
    # 이렇게 반복하여 추출한 좌표값을 반환
    return MET

MET = convert2_str_to_float(testset)
print(MET)

[[ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]]


In [8]:
img=skimage.io.imread(testset[0][0])
print(img.shape)
coor_set=convert_str_to_int(testset)
print(coor_set.shape)
print(coor_set[0][0])
print(coor_set[0][1])
print(coor_set[0][0][0])
print(coor_set[0][0][0]*(256/img.shape[0]))

(176, 79, 3)
(100, 14, 2)
[ 17 146]
[ 23 115]
17
24.7272727273


In [9]:
def Preprocess(dataset):
    img_set=np.zeros([len(dataset),256,256,3]) 
    # 이미지를 저장해두기 위한 텐서 배열 초기화 - 이미지 갯수 / x크기 / y크기 / RGB
    
    coor_set=convert_str_to_int(dataset) 
    # 문자열의 좌표값을 정수형 좌표값으로 변환하여 저장
    
    for i in range(len(dataset)): # 주어진 데이터 갯수만큼 반복
        img=skimage.io.imread(dataset[i][0]) # 이미지 불러오기
        
        img_set[i]=skimage.transform.resize(img,(256,256)) 
        # 이미지 크기를 재조정 하여 저장
        
        #이미지 크기 변환을 하였으므로 그에 걸맞게 좌표 위치또한 재조정
        for j in range(14): #주어진 좌표 개수 만큼 반복
            if coor_set[i][j][0]==-1: coor_set[i][j][0]=-1 
                # x 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][0]=coor_set[i][j][0]*(256/img.shape[1]) 
                # x축 크기의 변환된 비율 만큼 곱해 Scaling 
            if coor_set[i][j][1]==-1: coor_set[i][j][1]=-1 # y 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][1]=coor_set[i][j][1]*(256/img.shape[0]) # y축 크기의 변환된 비율 만큼 곱해 Scaling 
    
    # 이렇게 변환한 이미지 세트와 좌표 세트를 반환
    return img_set, coor_set

In [10]:
# 전처리 함수의 입력으로 불러온 CSV 파일을 저장한 testset 변수로 입력.
# 변환된 이미지 세트와 좌표 세트를 각각 picpic과 coco에다 저장해줌.
picpic,coco=Preprocess(testset)

/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [11]:
# 각 세트의 크기 확인
print (picpic.shape,coco.shape)

(100, 256, 256, 3) (100, 14, 2)


In [12]:
# 이미지와 좌표를 받아와서 이미에 해당 좌표를 입력
# 잘 변환이 되었는지 확인하는 용도

import cv2 # 그림에 숫자 넣기
from skimage import draw # 그림에 원 그리기

def markJoints(img, joints):  
    circSize=5 # 그려질 원의 크기
    font = cv2.FONT_HERSHEY_SIMPLEX # 글씨체 설정
    for i in range(14):
        x = int(joints[i,0]) # X 좌표
        y = int(joints[i,1]) # Y 좌표
        if x!=-1: # 그림에서 보이지 않는 좌표는 찍지 않도록
            rr, cc = skimage.draw.circle(y, x, circSize) # 원 그리기  `skimage.draw.circle(r, c, 반지름[shape])`
            skimage.draw.set_color(img, (rr, cc), (1,0,0)) # 색 설정 `set_color(image, coords, color)`
            cv2.putText(img, str(i+1), (x,y), font, 0.5, (0.5,0.5,0.5), 2, cv2.LINE_AA) # 그림 ㄱ
    return img


In [13]:
#imge = np.zeros((10,10), dtype=np.uint8)
#rr, cc = circle(4,4,5)
#imge[rr, cc]=1

In [14]:
# 전처리된 이미지 저장
%mkdir p_img p_with_coors_img # 현재 디렉토리에 처리된 이미지와 좌표를 찍은 이미지들을 저장할 디렉토리 생성
for i in range(len(picpic)):
    skimage.io.imsave("./p_img/p-img_"+str(i)+".jpg",picpic[i]) # 처리된 이미지 저장
    skimage.io.imsave("./p_with_coors_img/p-img_"+str(i)+".jpg",markJoints(picpic[i],coco[i])) # 좌표 입력 이미지 저장

mkdir: cannot create directory ‘p_img’: File exists
mkdir: cannot create directory ‘p_with_coors_img’: File exists


/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [15]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


## 1. Linear Regression

좌표 데이터 구조 변환 (100,14,2) -> (100,28)

[x1,y1,x2,y2 ..., x14,y14]의 구조형태

In [16]:
coco=np.reshape(coco,(100,28))
coco

array([[ 55, 212,  74, ...,  56, 113,  24],
       [ 93, 190,  51, ...,  56,  59,  23],
       [101, 217,  92, ...,  58,  88,  29],
       ..., 
       [ -1,  -1,  -1, ...,  57, 164,   0],
       [ -1,  -1,  63, ...,  62, 130,   8],
       [ -1,  -1,  -1, ...,  42, 152,   6]])

In [17]:
MET

array([[ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1

100개의 좌표와 MET데이터 중 10개를 test데이터로 사용

In [18]:
a=0;b=0
coco_train=np.zeros([90,28])
MET_train=np.zeros([90,1])
coco_test=np.zeros([10,28])
MET_test=np.zeros([10,1])

for i in range (100):
    if i % 10 == 0:
        coco_test[a]=coco[i]
        MET_test[a]=MET[i]
        a+=1
    else :
        coco_train[b]=coco[i]
        MET_train[b]=MET[i]
        b+=1

In [19]:
MET_test

array([[ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.8],
       [ 1.8]])

### Normal Equation

In [20]:
X_b = np.c_[np.ones((100, 1)), coco]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(MET)

In [21]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(coco_train, MET_train)
lin_reg.intercept_, lin_reg.coef_

(array([ 1.32554299]),
 array([[  9.81473262e-04,  -1.31981868e-03,  -2.48180814e-03,
           1.11351536e-03,   8.55150762e-04,  -1.28847674e-03,
          -1.00155038e-03,   2.29202113e-03,   1.40736435e-03,
          -1.95626547e-03,   9.04141842e-04,  -6.96280801e-04,
           7.57432273e-04,  -3.37373486e-04,   2.27710651e-03,
          -1.78510490e-04,  -6.92755158e-04,  -1.72643444e-04,
           3.71454295e-04,  -5.09212737e-03,  -2.96399183e-06,
           1.27724884e-04,  -4.59606132e-05,   1.42334697e-03,
          -9.37737028e-04,   4.80047730e-04,   1.78687719e-04,
           3.94785848e-03]]))

**score: 결정계수(coefficient of determination,R^2)를 보여줌, 독립적인 변수들로 부터의 분산 비율**

In [22]:
lin_reg.score(coco_train,MET_train)

0.39656911947210083

In [23]:
lin_reg.score(coco_test,MET_test)

0.097023759606756022

In [24]:
lin_reg.predict(coco_test)

array([[ 0.95956193],
       [ 1.05750781],
       [ 0.96811226],
       [ 0.7308875 ],
       [ 0.92192878],
       [ 1.75912466],
       [ 0.93470436],
       [ 1.03555068],
       [ 1.60127246],
       [ 1.18781324]])

### Linear Regression Conclusion

값이 좋진 않지만 어느정도 일정 범주에서 MET 값이 예측

데이터가 더 필요할 것으로 예상

# 2. SDG regression

In [25]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

In [26]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=50, penalty=None, eta0=0.1, random_state=42)
#sgd_reg.fit(X, y.ravel())
sgd_reg.fit(coco_train, MET_train.ravel())

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.1,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=50, n_iter=None, penalty=None,
       power_t=0.25, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [27]:
sgd_reg.intercept_, sgd_reg.coef_

(array([ -5.62245103e+11]),
 array([ -3.81168360e+12,   1.19802823e+12,   1.27409438e+13,
         -5.20516067e+12,  -1.06003007e+13,  -1.54081347e+13,
         -1.71887900e+12,  -1.84994703e+12,   4.54242259e+11,
         -2.79765475e+12,  -7.57524344e+12,   1.52952900e+12,
         -5.10977176e+12,  -6.57261023e+12,  -6.63205100e+12,
         -1.44799217e+12,   4.25016915e+12,   6.97489956e+12,
          3.63762082e+12,   1.11570556e+13,  -1.60554628e+12,
          3.60202724e+12,  -9.74488726e+12,   5.15424488e+12,
          1.64762264e+11,   2.90162020e+12,   2.60479766e+12,
          9.22233278e+12]))

In [28]:
sgd_reg.predict(coco_test)

array([ -2.49444234e+15,  -4.99741175e+15,  -1.73670179e+13,
         3.92314121e+15,  -6.04116546e+13,  -9.84963345e+14,
        -5.25743407e+15,  -3.65387994e+15,  -6.15415994e+15,
        -4.19793686e+15])

# 3. Polynomial Regression

In [29]:

from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(coco_train)

In [30]:
X_poly[0]

array([  9.30000000e+01,   1.90000000e+02,   5.10000000e+01,
         1.71000000e+02,   9.80000000e+01,   1.25000000e+02,
         1.49000000e+02,   1.25000000e+02,   1.23000000e+02,
         1.71000000e+02,   1.49000000e+02,   2.07000000e+02,
        -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00,
        -1.00000000e+00,   5.10000000e+01,   6.50000000e+01,
         1.57000000e+02,   6.00000000e+01,   2.04000000e+02,
         8.10000000e+01,   2.17000000e+02,   1.06000000e+02,
         8.90000000e+01,   5.60000000e+01,   5.90000000e+01,
         2.30000000e+01,   8.64900000e+03,   1.76700000e+04,
         4.74300000e+03,   1.59030000e+04,   9.11400000e+03,
         1.16250000e+04,   1.38570000e+04,   1.16250000e+04,
         1.14390000e+04,   1.59030000e+04,   1.38570000e+04,
         1.92510000e+04,  -9.30000000e+01,  -9.30000000e+01,
        -9.30000000e+01,  -9.30000000e+01,   4.74300000e+03,
         6.04500000e+03,   1.46010000e+04,   5.58000000e+03,
         1.89720000e+04,

In [31]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, MET_train)
lin_reg.intercept_, lin_reg.coef_

(array([ 1.94021047]),
 array([[ -1.67415131e-07,  -2.54258118e-07,  -1.79362961e-07,
          -2.17809020e-07,  -7.55433788e-08,  -7.48909203e-08,
          -1.13725107e-07,  -8.88311804e-08,  -2.27420598e-07,
          -2.81408060e-07,  -2.16205213e-07,  -2.96459036e-07,
          -4.22694988e-08,  -2.68106032e-08,  -1.81291774e-08,
          -1.05073614e-08,   3.35688866e-10,  -1.45809123e-08,
           1.90164065e-08,   1.49777016e-08,   3.26264492e-08,
          -6.20607325e-10,   4.89305390e-08,  -1.90628709e-08,
          -1.19668227e-08,  -1.44636719e-08,  -3.18769573e-08,
          -2.12573554e-08,  -1.81867693e-05,  -1.07162347e-06,
          -9.78858386e-06,   2.63907392e-06,   3.78972997e-06,
           1.90644375e-06,   7.89846981e-06,  -1.25890454e-06,
           1.00468954e-05,   4.75849622e-06,   9.61948109e-06,
           4.41242842e-06,  -9.88343264e-07,  -5.11888460e-06,
          -2.86765582e-07,   1.30203863e-06,   3.43533906e-06,
           3.11327311e-06,   3.8

In [32]:
lin_reg.predict(coco_test)

ValueError: shapes (10,28) and (434,1) not aligned: 28 (dim 1) != 434 (dim 0)

### Polynomial Regression Conclusion

오류발생으로 해결 못한 상태

# 4. MLPRegression

In [33]:
import numpy as np
from sklearn.neural_network import MLPRegressor 

X = coco_train
y = MET_train.astype(np.float)

per_clf = MLPRegressor(random_state=42, activation="relu", solver="adam")
per_clf.fit(X, y)

y_pred = per_clf.predict(coco_test)

/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [34]:
MET_test

array([[ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.8],
       [ 1.8]])

In [35]:
y_pred

array([  5.82814589,   8.49427969,  -3.60274039,   7.23754122,
        -4.46605475,  19.96264307,  10.91469327,   6.35614281,
        -5.68677949, -12.04871961])

# 5. TensorFlow

In [36]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [37]:
import tensorflow as tf

n_inputs = 28 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 1     #MET값 1개

In [38]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [39]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [40]:
X.get_shape()[1]

Dimension(28)

In [41]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons], dtype=tf.float32), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [42]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")


In [43]:
from sklearn.metrics import mean_squared_error

with tf.name_scope("loss"):
    #xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
    #                                                          logits=logits)
    loss = tf.losses.mean_squared_error(y, logits)
    #loss = -tf.reduce_sum(logits * tf.log(p))
    #loss = tf.reduce_mean(xentropy, name="loss")

In [44]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [45]:
#with tf.name_scope("eval"):
#    correct = tf.nn.in_top_k(logits, y, 1)  -> 위에서부터 가장 큰 값만 찾는건 라벨링 된 이미지 처리할때
#    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [46]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [47]:
n_epochs = 500

In [48]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: coco_train, y: MET_train})
        #acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
        #                                    y: mnist.validation.labels})
        print(epoch, "loss :", loss)

    save_path = saver.save(sess, "./dnn0319.ckpt")

0 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
1 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
2 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
3 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
4 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
5 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
6 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
7 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
8 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
9 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
10 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
11 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
12 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
13 loss :

117 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
118 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
119 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
120 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
121 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
122 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
123 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
124 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
125 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
126 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
127 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
128 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
129 loss : Tensor("loss/mean_squared_error/value:0", shape=(), d

282 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
283 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
284 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
285 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
286 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
287 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
288 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
289 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
290 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
291 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
292 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
293 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
294 loss : Tensor("loss/mean_squared_error/value:0", shape=(), d

449 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
450 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
451 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
452 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
453 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
454 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
455 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
456 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
457 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
458 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
459 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
460 loss : Tensor("loss/mean_squared_error/value:0", shape=(), dtype=float32)
461 loss : Tensor("loss/mean_squared_error/value:0", shape=(), d

In [49]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = coco_test
    y_pred = logits.eval(feed_dict={X: X_new_scaled})
    #y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [50]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", MET_test)

Predicted classes: [[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]
Actual classes:    [[ 0.7]
 [ 0.7]
 [ 1. ]
 [ 1. ]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.8]
 [ 1.8]]


### TensorFlow conclusion (현재 진행 상황)
    
DNN 학습할때, 기존 MNIST 처럼 이미지 label 분류가 아닌 MET 값을 도출해야 해서

Loss는 softmax가 아닌 mean_square_error를 사용 -> Loss값 보는 방법 개선 필요..

correct와 accuracy 부분은 아직 해결책을 못찾음

결과적으로 값으로 예측하기는 실패